In [1]:
import os
import cv2
import dlib
import pandas as pd

# Paths
input_folder = "./images"  # Replace with the path to your folder containing images
output_csv = "features_dataset.csv"

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Helper function to extract distances between landmarks
def extract_features(shape):
    # Key landmarks
    left_eye = shape[36]
    right_eye = shape[45]
    nose_tip = shape[33]
    left_mouth = shape[48]
    right_mouth = shape[54]
    chin = shape[8]
    left_jaw = shape[0]
    right_jaw = shape[16]

    # Feature calculations (distances in pixel units)
    features = {
        "eye_distance": ((right_eye[0] - left_eye[0]) ** 2 + (right_eye[1] - left_eye[1]) ** 2) ** 0.5,
        "nose_to_chin": ((chin[0] - nose_tip[0]) ** 2 + (chin[1] - nose_tip[1]) ** 2) ** 0.5,
        "mouth_width": ((right_mouth[0] - left_mouth[0]) ** 2 + (right_mouth[1] - left_mouth[1]) ** 2) ** 0.5,
        "jaw_width": ((right_jaw[0] - left_jaw[0]) ** 2 + (right_jaw[1] - left_jaw[1]) ** 2) ** 0.5,
        "nose_to_eye": ((nose_tip[0] - left_eye[0]) ** 2 + (nose_tip[1] - left_eye[1]) ** 2) ** 0.5,
    }

    return features

# Iterate over images and process
features_list = []
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)
    if len(faces) > 0:
        # Assume the first detected face
        face = faces[0]
        landmarks = predictor(gray, face)

        # Convert landmarks to a list of tuples
        shape = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

        # Extract features
        features = extract_features(shape)
        features["image_name"] = image_file
        features_list.append(features)
    else:
        print(f"No face detected in {image_file}")

# Save features to CSV
df = pd.DataFrame(features_list)
df.to_csv(output_csv, index=False)

print(f"Feature extraction complete. Data saved to {output_csv}")


No face detected in image56.jpg
No face detected in image57.jpg
No face detected in image64.jpg
No face detected in image84.jpg
No face detected in image89.jpg
No face detected in image90.jpg
No face detected in image92.jpg
Feature extraction complete. Data saved to features_dataset.csv
